In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
!pip install catboost
path = "/kaggle/input/spaceship-titanic/"

In [ ]:
train = pd.read_csv(path+"train.csv")
test = pd.read_csv(path+"test.csv")
train

In [ ]:
train = train[train['Cabin'].notna()]
train

In [ ]:
train["CryoSleep"] = list(map(lambda x: 1 if x or x==1 else 0,list(train["CryoSleep"])))
train["VIP"] = list(map(lambda x: 1 if x or x==1 else 0,list(train["VIP"])))

test["CryoSleep"] = list(map(lambda x: 1 if x or x==1 else 0,list(test["CryoSleep"])))
test["VIP"] = list(map(lambda x: 1 if x or x==1 else 0,list(test["VIP"])))
train

In [ ]:
def homePlanetToInt(planet):
  if (planet=="Europa"): return 1
  elif (planet=="Earth"): return 2
  elif (planet=="Mars"): return 3
  else: return 4

In [ ]:
train["HomePlanet"] = list(map(lambda x: homePlanetToInt(x),list(train["HomePlanet"])))
test["HomePlanet"] = list(map(lambda x: homePlanetToInt(x),list(test["HomePlanet"])))
train

In [ ]:
def IDToGroup(id):
  id = id.split("_")[0]
  return int(id)

In [ ]:
train["Group"] = list(map(lambda x: IDToGroup(x),list(train["PassengerId"])))
test["Group"] = list(map(lambda x: IDToGroup(x),list(test["PassengerId"])))
train

In [ ]:
def IDToGroupNumber(id):
  return int(id.split("_")[1])

In [ ]:
train["GroupNumber"] = list(map(lambda x: IDToGroupNumber(x),list(train["PassengerId"])))
test["GroupNumber"] = list(map(lambda x: IDToGroupNumber(x),list(test["PassengerId"])))

In [ ]:
def destinationToInt(dest):
  if (dest=="TRAPPIST-1e"): return 1
  elif (dest=="55 Cancri e"): return 2
  else: return 3

In [ ]:
train["Destination"] = list(map(lambda x: destinationToInt(x),list(train["Destination"])))
test["Destination"] = list(map(lambda x: destinationToInt(x),list(test["Destination"])))

In [ ]:
def cabinToDeck(cabin):
  data = cabin.split("/")
  return (ord(data[0])-65)

In [ ]:
train["Deck"] = list(map(lambda x: cabinToDeck(x),list(train["Cabin"].fillna("A/0/P"))))
test["Deck"] = list(map(lambda x: cabinToDeck(x),list(test["Cabin"].fillna("A/0/P"))))

In [ ]:
def cabinToRoomNum(cabin):
  return int(cabin.split("/")[1])

In [ ]:
train["CabinNum"] = list(map(lambda x: cabinToRoomNum(x),list(train["Cabin"].fillna("A/0/P"))))
test["CabinNum"] = list(map(lambda x: cabinToRoomNum(x),list(test["Cabin"].fillna("A/0/P"))))

In [ ]:
def cabinToSide(cabin):
  side = cabin.split("/")[2]
  return 0 if side=="P" else 1

In [ ]:
train["Side"] = list(map(lambda x:cabinToSide(x),list(train["Cabin"].fillna("A/0/P"))))
test["Side"] = list(map(lambda x: cabinToSide(x),list(test["Cabin"].fillna("A/0/P"))))

In [ ]:
train

In [ ]:
x_train = np.column_stack([train["CryoSleep"],
                           train["VIP"],
                           train["Age"].fillna(np.mean(train["Age"])),
                           train["RoomService"].fillna(np.mean(train["RoomService"])),
                           train["FoodCourt"].fillna(np.mean(train["FoodCourt"])),
                           train["ShoppingMall"].fillna(np.mean(train["ShoppingMall"])),
                           train["Spa"].fillna(np.mean(train["Spa"])),
                           train["VRDeck"].fillna(np.mean(train["VRDeck"])),
                           train["HomePlanet"],
                           train["Group"],
                           train["GroupNumber"],
                           train["Destination"],
                           train["Deck"],
                           train["CabinNum"],
                           train["Side"]])
y_train = list(map(lambda x: int(x),train["Transported"]))

In [ ]:
from sklearn.model_selection import train_test_split
x_test = np.column_stack([test["CryoSleep"],
                          test["VIP"],
                          test["Age"].fillna(np.mean(test["Age"])),
                          test["RoomService"].fillna(np.mean(test["RoomService"])),
                          test["FoodCourt"].fillna(np.mean(test["FoodCourt"])),
                          test["ShoppingMall"].fillna(np.mean(test["ShoppingMall"])),
                          test["Spa"].fillna(np.mean(test["Spa"])),
                          test["VRDeck"].fillna(np.mean(test["VRDeck"])),
                          test["HomePlanet"],
                          test["Group"],
                          test["GroupNumber"],
                          test["Destination"],
                          test["Deck"],
                          test["CabinNum"],
                          test["Side"]])
#x_train,x_test,y_train,y_test = train_test_split(x_train,y_train,test_size=.2)
#y_test = list(map(lambda x: int(x),y_test))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
#Cat Boost Parameters Found by Igor Shirokov
#https://www.kaggle.com/code/igorshirokov/top-10-space-titanic
m = VotingClassifier([("Gradient",GradientBoostingClassifier(criterion='squared_error', loss= 'deviance', max_features='auto')),
                     ("Cat",CatBoostClassifier(depth=6,subsample=.4,colsample_bylevel=.28,random_strength=.5355,l2_leaf_reg=4.9,min_data_in_leaf=24,verbose=0))],
                     voting="hard")
m.fit(x_train,y_train)
p = m.predict(x_test)

In [ ]:
p = list(map(lambda x: bool(x),p))
submission = pd.DataFrame({"PassengerId":list(test["PassengerId"]),"Transported":p})
submission.to_csv("submission.csv",index=False)
submission